<a href="https://colab.research.google.com/github/bebechien/gemma/blob/main/Game_Design_Brainstorming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemma - Game Design Brainstorming

This cookbook illustrates how you can employ the instruction-tuned model version of Gemma as a chatbot to aid in your game design process by fostering brainstorming sessions effectively.

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a A100 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **A100 GPU**.


### Gemma setup on Kaggle
To complete this tutorial, you'll first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

* Get access to Gemma on kaggle.com.
* Select a Colab runtime with sufficient resources to run the Gemma 2B model.
* Generate and configure a Kaggle username and API key.

After you've completed the Gemma setup, move on to the next section, where you'll set environment variables for your Colab environment.

### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

Install Keras and KerasNLP.

In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

## Core Concept

In [3]:
# A super concise description of your game
pitch = "In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution."

## Load Model

In [4]:
import keras
import keras_nlp

# Run at half precision to fit in memory
keras.config.set_floatx("bfloat16")

import textwrap
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_1.1_instruct_7b_en")
gemma_lm.compile(sampler="top_k")

class GemmaChat():

  __START_TURN__ = "<start_of_turn>"
  __END_TURN__ = "<end_of_turn>"

  def __init__(self, model, system="", history=None):
    self.model = model
    self.system = system
    if not history:
      self.history = []
    else:
      self.history = history

  def format_message(self, message, prefix=""):
    return f"{self.__START_TURN__}{prefix}\n{message}{self.__END_TURN__}\n"

  def add_to_history(self, message, prefix=""):
    formated_message = self.format_message(message, prefix)
    self.history.append(formated_message)

  def get_full_prompt(self):
    prompt = "\n".join([self.system + "\n", *self.history])
    return prompt

  def send_message(self, message):
    self.add_to_history(message, "user")
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    # print('--------------------response---------------------------')
    # print(response)
    # print('--------------------response---------------------------')
    result = response.replace(prompt, "")
    self.add_to_history(result, "model")
    return result

  def show_history(self):
    for h in self.history:
      print(h)

chat = GemmaChat(gemma_lm)


100%|██████████| 556/556 [00:00<00:00, 696kB/s]
100%|██████████| 15.9G/15.9G [03:57<00:00, 71.8MB/s]
100%|██████████| 401/401 [00:00<00:00, 605kB/s]
100%|██████████| 4.04M/4.04M [00:00<00:00, 17.1MB/s]


## Elaborate with the given core game concept

In [5]:
result = chat.send_message(f"Elaborate about this game with the given core concept below.\n{pitch}")
to_markdown(result)

> **Core Concept:**
> * **Steampunk meets Joseon Dynasty:** A fusion of Victorian-era technology with the rich cultural setting of Joseon Dynasty Korea.
> * **Mystery & Exploration:** Unearthing forgotten secrets of the past through exploration, puzzle-solving, and interactions with eccentric characters.
> * **Choice & Consequences:** Shaping the fate of the nation through decisions made in dialogue and action.
> * **Political Intrigue:** Uncovering conspiracies and power struggles within the royal court.
> 
> **Additional Elements:**
> * A charismatic protagonist with a shrouded past.
> * A diverse cast of characters, both loyal allies and cunning enemies.
> * Moral dilemmas and ethical questions surrounding technological advancement.
> * High-octane action sequences and daring escapes.
> 
> **Possible Gameplay Elements:**
> * Puzzle-solving and interactive environments.
> * Stealth and action combat.
> * Character customization and skill trees.
> * Reputation system and branching storylines.
> 
> **Target Audience:**
> * Fans of historical settings and Asian culture.
> * Players who enjoy mystery and puzzle-solving games.
> * Anyone who loves stories with political intrigue and high stakes.
> 
> **Possible Title:**
> * The Clockwork Emperor
> * Shadow of the Shogun
> * Crimson Sky Chronicles
> 
> **Overall Tone:**
> * Epic and cinematic.
> * Mysterious and atmospheric.
> * Suspenseful and action-packed.
> 
> 
> **Additional Notes:**
> * The setting should be rich in detail, with a focus on steam-powered machines and clockwork devices.
> * The story should be full of twists and turns, keeping players engaged.
> * The game should provide multiple endings, depending on the player's choices and actions.
> 
> I hope this elaboration gives you a clearer understanding of the game concept. Please let me know if you have any further questions or require additional details.

## Main Characters

In [6]:
result = chat.send_message("Design main characters.")
to_markdown(result)

> ## Main Character Concepts:
> 
> **1. Jin-ah, the Master Engineer:**
> 
> * A skilled inventor and engineer, driven by curiosity and a yearning to improve the lives of the people.
> * Possesses a vast knowledge of steam-powered technology and clockwork mechanisms.
> * Haunted by a failed experiment that left her with mysterious scars.
> 
> 
> **2. Prince Hyon, the Hidden Rebel:**
> 
> * A charismatic prince who secretly opposes the oppressive government.
> * Skilled in martial arts and politics.
> * Holds a deep connection to the Joseon Dynasty's traditions and values.
> 
> 
> **3. Lady Mi-jin, the Royal Oracle:**
> 
> * A cunning strategist and political manipulator.
> * Possesses supernatural abilities and a vast network of informants.
> * Holds the key to unlocking the secrets of the forgotten past.
> 
> 
> **4. General Jang, the Loyal Protector:**
> 
> * A skilled military strategist and loyal bodyguard.
> * Known for his unwavering loyalty and unwavering sense of justice.
> * Hides a hidden vulnerability beneath his stoic exterior.
> 
> 
> **5. The Whispering Machine:**
> 
> * A mysterious and sentient steam-powered automaton.
> * Claims to possess knowledge of a lost technology that could change the course of history.
> * Offers cryptic warnings and guidance to the protagonist.

## Villain Characters

In [7]:
result = chat.send_message("Design villain characters.")
to_markdown(result)

> ## Villain Concepts:
> 
> **1. The Shadow Chancellor:**
> 
> * A cunning politician who manipulates the Emperor from the shadows.
> * Obsessed with maintaining his power and will stop at nothing to achieve his goals.
> * Secretly working to suppress technological advancements and maintain the existing social order.
> 
> 
> **2. The Clocksmith:**
> 
> * A ruthless inventor who seeks to dominate the world with his advanced clockwork technology.
> * Driven by a desire to control others and create a society where efficiency reigns supreme.
> * Possesses a twisted vision of a future where humanity is enslaved by machines.
> 
> 
> **3. The Serpent Oracle:**
> 
> * A vengeful spirit who was wronged by the Joseon Dynasty centuries ago.
> * Returns to exact revenge on those who have oppress her people.
> * Uses her mystical knowledge and manipulative abilities to sow chaos and destruction.
> 
> 
> **4. The Crimson Alchemist:**
> 
> * A mad scientist who seeks to harness the power of steam to achieve immortality.
> * His experiments are destructive and dangerous, threatening the entire nation.
> * Driven by his insatiable thirst for knowledge and his desire to defy the limits of human existence.
> 
> 
> **5. The Wandering Shadow:**
> 
> * A mysterious assassin who targets those who stand in the way of the villain's plans.
> * Possesses an unmatched skill in stealth and assassination.
> * Remains shrouded in mystery and leaves behind cryptic clues.

## Main Story

In [8]:
result = chat.send_message("Write the main story of this game with an introduction, development, turn, and conclusion.")
to_markdown(result)

> ## The Crimson Legacy
> 
> **Introduction:**
> 
> In the year 1899, the Joseon Dynasty stands on the precipice of a new age. Steam-powered machines and clockwork contraptions have revolutionized every aspect of life, but their advancement casts a shadow of concern over the nation. In the heart of this technological upheaval lies a forgotten past, filled with secrets that could reshape the future.
> 
> **Development:**
> 
> Jin-ah, a master engineer, stumbles upon a secret research facility hidden beneath the Imperial Palace. There, she discovers a hidden archive that reveals a forgotten history of steam-powered technology that predates the present era. As she delves deeper, she attracts the attention of both the Emperor's loyal protectors and a sinister cabal of political elites who seek to control the technology for their own agendas.
> 
> Meanwhile, Prince Hyon, heir to the throne, discovers that his lineage has a connection to the lost technology. Driven by his sense of duty, he joins Jin-ah in her investigation, knowing that the fate of the nation rests in their hands.
> 
> **Turn:**
> 
> As Jin-ah and Hyon uncover the secrets of the past, they find themselves targeted by the Shadow Chancellor, who seeks to maintain his control. The villain manipulates the court, unleashing monstrous clockwork creatures and unleashing chaos across the land. The friends must grapple with political intrigue, desperate battles, and moral dilemmas.
> 
> **Conclusion:**
> 
> Jin-ah and Hyon must confront the Shadow Chancellor and his allies, utilizing their combined skills and knowledge to unlock the true potential of steam-powered technology. Their actions will determine the destiny of the Joseon Dynasty. Will they restore balance to the nation or will they forge a new future built on the foundation of forgotten knowledge?

## Show the chat history

In [9]:
chat.show_history()

<start_of_turn>user
Elaborate about this game with the given core concept below.
In the heart of a reimagined Joseon Dynasty, where steam-powered machines and clockwork contraptions reign supreme, embark on an epic adventure to unravel the mysteries of a forgotten past and shape the destiny of a nation on the brink of revolution.<end_of_turn>

<start_of_turn>model
**Core Concept:**
* **Steampunk meets Joseon Dynasty:** A fusion of Victorian-era technology with the rich cultural setting of Joseon Dynasty Korea.
* **Mystery & Exploration:** Unearthing forgotten secrets of the past through exploration, puzzle-solving, and interactions with eccentric characters.
* **Choice & Consequences:** Shaping the fate of the nation through decisions made in dialogue and action.
* **Political Intrigue:** Uncovering conspiracies and power struggles within the royal court.

**Additional Elements:**
* A charismatic protagonist with a shrouded past.
* A diverse cast of characters, both loyal allies and cu

## Clear History

In [10]:
chat.history.clear()

## Expansion Idea

### Marketing Phrase

In [11]:
chat.history.clear()
pitch = "A new steam-powered toothbrush"
result = chat.send_message(f"Generate a marketing phrase for the new product below.\n{pitch}")
to_markdown(result)

> * Cleans teeth with pure water and natural steam
> * Reduces plaque and promotes gum health
> * Provides a deep clean without harsh chemicals or abrasives
> 
> **Marketing Phrase Options:**
> 
> - **Steam to a Brighter Smile**
> - **Freshness Unleashed, Naturally**
> - **Clean Teeth, Naturally Powered**
> - **The Gentle Revolution in Oral Care**
> - **Pure Clean, Pure Health**
> 
> **Which marketing phrase would you choose and why?**
> 
> I would choose **"Pure Clean, Pure Health"** because it clearly communicates the product's unique benefits - a deep clean without the use of harsh chemicals or abrasives, resulting in improved oral health. The phrase is concise, memorable, and emphasizes the natural and effective approach to oral care.

### Florist Ideas

In [12]:
chat.history.clear()
pitch = "Universe and shooting stars"
result = chat.send_message(f"Generate a florist idea inspired by the concept below, along with suggestions for suitable flowers.\n{pitch}")
to_markdown(result)

> **Florist Idea:**
> 
> **Concept:** Capture the wonder of the universe and the fleeting beauty of shooting stars in a breathtaking floral arrangement. 
> 
> **Suggested Flowers:**
> 
> - **Roses:** represent love, beauty, and new beginnings, symbolizing the vastness of space.
> - **Stargazers:** resemble shooting stars with their elongated stems and star-like heads, adding an ethereal touch. 
> - **Hydrangeas:** represent growth, rebirth, and transformation, reflecting the cosmic evolution.
> - **Delphiniums:** symbolize freedom and imagination, connecting to the boundless possibilities of the universe.
> - **Waxflowers:** represent purity and innocence, reminiscent of stardust.
> 
> 
> **Additional Elements:**
> 
> - Silver or metallic foliages to evoke the gleam of stars.
> - Crystals or iridescent stones to capture the sparkle and glow of the cosmos.
> - A clear glass vase with star-shaped cutouts, allowing glimpses of the universe through the arrangement.
> 
> 
> **Display Ideas:**
> 
> - **Suspended arrangement:** Hang the flowers from the ceiling, creating a floating cloud of cosmic beauty.
> - **Table centerpiece:** Arrange the flowers in a geometric pattern on a mirrored surface, reflecting the starry sky.
> - **Gothic vase arrangement:** Fill a vintage glass vase with a mix of blooms, adding an ethereal and otherworldly touch to any space.

### Food Recipe

In [13]:
chat.history.clear()
pitch = "Cyberpunk Kraken"
result = chat.send_message(f"Generate a cooking recipe with the concept below.\n{pitch}")
to_markdown(result)

> A refreshing cocktail inspired by the deep oceans and futuristic technology.
> 
> **Concept:**
> - Base: Cucumber infused aquavit
> - Flavor profile: Sea brine, kelp, seaweed, lime, and a touch of futuristic technology with activated charcoal and blue light.
> 
> 
> **Recipe:**
> 
> **Ingredients:**
> 
> - 2 oz Cucumber infused aquavit
> - 2 oz Lime juice, freshly squeezed
> - 1/2 oz Activated charcoal powder (dissolved in water)
> - 1/4 cup Kelp infusion
> - 1/8 cup Sea salt brine
> - Blue light for garnish
> 
> 
> **Instructions:**
> 
> 1. In a cocktail shaker filled with ice, combine the cucumber infused aquavit, lime juice, activated charcoal solution, kelp infusion, and sea salt brine.
> 
> 
> 2. Shake vigorously for about 10 seconds until chilled and well blended.
> 
> 
> 3. Strain into a chilled martini glass.
> 
> 
> 4. As a finishing touch, expose the cocktail to blue light for a few moments to create an ethereal glow.
> 
> 
> **Garnish:**
> 
> - A sprig of fresh seaweed
> - A lime twist
> 
> **Tips:**
> 
> - For the cucumber infused aquavit, infuse aquavit with sliced cucumbers, seaweed, and sea salt for a few days.
> 
> 
> - Activated charcoal adds a unique smoky flavor and a touch of filtration. Be sure to dissolve the charcoal powder in water before adding it to the cocktail.
> 
> 
> - Kelp infusion can be made by steeping kelp in boiling water for a few minutes.
> 
> 
> - Sea salt brine is simply sea salt dissolved in water.